# Automated ML

Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [75]:
from azureml.core import Workspace, Experiment
from azureml.train.automl import AutoMLConfig

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails

from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core import Environment

import joblib

import requests

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [4]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'amazon_review_prediction_automl'

experiment=Experiment(ws, experiment_name)

found = False
dataset_name = "Amazon Reviews"
dataset_descriptions = "Set of Amazon products text reviews together with their rating value ranging from 1 to 5."

if dataset_name in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[dataset_name] 

if not found:
        dataset = TabularDatasetFactory.from_delimited_files(path="https://github.com/iolucas/AMLE_P3/raw/main/amazon_reviews.csv") 
        dataset = ds.register(workspace=ws, name=dataset_name, description=dataset_descriptions)

In [11]:
amlcompute_cluster_name = "compute-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS12_v2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

Found existing cluster, use it.


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [17]:
automl_settings = {
    "experiment_timeout_minutes": 30,
    "task": "classification",
    "primary_metric": "accuracy",
    "training_data": dataset,
    "label_column_name": "rating",
    "n_cross_validations":5,
    "compute_target": compute_target,
    "max_concurrent_iterations": 4,
    "iteration_timeout_minutes": 5,
    "iterations": 25    
}

automl_config = AutoMLConfig(**automl_settings)

In [18]:
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
AutoML-Exp-P3,AutoML_41b227b8-6de8-4d19-9b0f-0463a621bc87,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [22]:
RunDetails(remote_run).show()
remote_run.wait_for_completion()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

{'runId': 'AutoML_41b227b8-6de8-4d19-9b0f-0463a621bc87',
 'target': 'compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-08-15T14:19:22.091836Z',
 'endTimeUtc': '2021-08-15T14:59:41.172059Z',
 'properties': {'num_iterations': '25',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'compute-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"AutoML-Exp-P3","subscription_id":"9a7511b8-150f-4a58-8528-3e7d50216c31","resource_group":"aml-quickstarts-154595","workspace_name":"quick-starts-ws-154595","region":"southcentralus","compute_target":"compute-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":25,"primary_metric":"accu

## Best Model

In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [54]:
best_run = remote_run.get_best_child()
best_run_metrics = best_run.get_metrics()

print('Best Run Id: ', best_run.id)

print('\n Accuracy:', best_run_metrics['accuracy'])

Best Run Id:  AutoML_41b227b8-6de8-4d19-9b0f-0463a621bc87_23

 Accuracy: 0.8632642235928356


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

In the cell below, register the model, create an inference config and deploy the model as a web service.

In [67]:
best_run.download_file("outputs/model.pkl")

In [68]:
from azureml.core import Model

model = Model.register(workspace=ws,
                       model_name='best_model_automl',                # Name of the registered model in your workspace.
                       model_path='./model.pkl',  # Local file to upload and register as a model.
                      )

Registering model best_model_automl


In [74]:
automl_env = Environment.get(ws, "AzureML-AutoML")
inference_config = InferenceConfig(source_directory=".", entry_script="score.py", environment=automl_env)
aci_config = AciWebservice.deploy_configuration(cpu_cores=2, memory_gb=2)

service_name = 'automl-model'

service = model.deploy(ws, service_name, [model], overwrite=True, inference_config=inference_config, deployment_config=aci_config)
service.wait_for_deployment(show_output=True)

NameError: name 'AciWebservice' is not defined

In the cell below, send a request to the web service you deployed to test it.

In [73]:
reviews = [
    "i loved it", 
    "fine product",
    "I hate it",
    "awful experience"
]

predictions = requests.post(service.scoring_uri, json={
    "data": reviews
}).json()

pd.DataFrame(data={"reviews":reviews, "Predicted Rating":predictions})

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/anaconda/envs/azureml_py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-73-546e7f9f487b>", line 9, in <module>
    "data": reviews
  File "/anaconda/envs/azureml_py36/lib/python3.6/site-packages/requests/api.py", line 119, in post
    return request('post', url, data=data, json=json, **kwargs)
  File "/anaconda/envs/azureml_py36/lib/python3.6/site-packages/requests/api.py", line 61, in request
    return session.request(method=method, url=url, **kwargs)
  File "/anaconda/envs/azureml_py36/lib/python3.6/site-packages/requests/sessions.py", line 542, in request
    resp = self.send(prep, **send_kwargs)
  File "/anaconda/envs/azureml_py36/lib/python3.6/site-packages/requests/sessions.py", line 655, in send
    r = adapter.send(request, **kwargs)
  File "/anaconda/envs/azureml_py36/lib/python3.6/site-packages/requests/adapters.py"

TypeError: object of type 'NoneType' has no len()

In the cell below, print the logs of the web service and delete the service

In [ ]:
print(service.get_logs())

In [ ]:
#service.delete()